# HTTP Request/Response Cycle - Lab

## Introduction 

In this lab, we'll make use of the `requests` module commands and properties seen in the previous lesson, to extract information for a web service called **"Open Notify"** to access NASA's space data. 

## Objectives

You will be able to:

* Explain the HTTP request/response cycle
* List the status codes of responses and their meanings
* Obtain and interpret status codes from responses
* Make HTTP GET and POST requests in python using the `requests` library

## Open Notify 

[Open Notify](http://open-notify.org/)  is an an open source project to provide a simple programming interface for some of NASA’s awesome data. This takes live raw data from NASA's systems and turn them into APIs related to space and spacecraft. We can access the following information from open notify. 

* Current Location of the International Space Station

* Overhead Pass Predictions for the International Space Station

* Number of People in Space
    
### API endpoints

OpenNotify has several API endpoints. 
>An endpoint is a server route that is used to retrieve different data from the API. 

For example, the `/comments` endpoint on the Reddit API might retrieve information about comments, whereas the `/users` endpoint might retrieve data about users. To access them, you would add the endpoint to the base url of the API.

For the OpenNotify API, we have following end points 

1. Current Location of the International Space Station `/iss-now.json`
2. Overhead Pass Predictions for the International Space Station `/iss-pass.json`    
3. Number of People in Space `/astros.json`

The `.json` extension simply tells us that the data is being returned in a JSON format.

In this lab, we'll be querying this API to retrieve live data about the International Space Station (ISS). Details on OpenNofity, endpoints, syntax, and the services it offers can be viewed [Here](http://open-notify.org/Open-Notify-API/)

![](images/iss.jpg)

### Current location of International Space Station

The first endpoint we'll look at on OpenNotify is the ` iss-now.json` endpoint (current location of international space station). This endpoint gets the current latitude and longitude of the International Space Station.  Perform the following tasks 
* Make a get request to get the latest position of the international space station from the opennotify api's `iss-now` endpoint at http://api.open-notify.org/iss-now.json
* Check the status code of the response
* Interpret the returned code

In [2]:
# Importing libraries
import requests
import json
import pandas as pd

In [10]:
# Getting response and checking status
resp = requests.get("http://api.open-notify.org/iss-now.json")
if resp.status_code == requests.codes.ok:
    print("Status OK")
else:
    print("Didn't Work")

Status OK


The response has come through ok.

* Print the contents of the response and identify its current location

In [11]:
# Printing text and headers
print(resp.text)
print(dict(resp.headers))

{"message": "success", "iss_position": {"latitude": "0.7487", "longitude": "-31.2364"}, "timestamp": 1575676733}
{'Server': 'nginx/1.10.3', 'Date': 'Fri, 06 Dec 2019 23:58:53 GMT', 'Content-Type': 'application/json', 'Content-Length': '112', 'Connection': 'keep-alive', 'access-control-allow-origin': '*'}


In [12]:
# Interpret your results using the APIa

At the time of the response the ISS was at "latitude": "0.7487", "longitude": "-31.2364"

### Check the next pass of International space station for a given location

Let's repeat the above for the second endpoint `iss-pass.json`. This end point is used to query the next pass of the space station on a given location. Let's just run as above and record your observations.

In [15]:
# Getting response and checking status
resp2 = requests.get("http://api.open-notify.org/iss-pass.json")
if resp2.status_code == requests.codes.ok:
    print("Status OK")
else:
    print("Didn't Work")

Didn't Work


In [16]:
# Your comments 
# SOMETHING IS WRONG!

So clearly there is something wrong as we had a 400 response. This is how you should always test your responses for validity. 

if we look at the documentation for the OpenNotify API, we see that the ISS Pass endpoint requires two parameters.

> The ISS Pass endpoint returns when the ISS will next pass over a given location on earth. In order to compute this, we need to pass the coordinates of the location to the API. We do this by passing two parameters -- latitude and longitude.

We can do this by adding an optional keyword argument, params, to our request. In this case, there are two parameters we need to pass:

* lat -- The latitude of the location we want.
* lon -- The longitude of the location we want.

Perform the following tasks :
* Set parameters to reflect the lat and long of New York  (40.71, -74)
* Send a get request to OpenNotify passing in the lat long parameters as k:v pairs in a dictionary
* Check the status code and interpret
* Print the header information and the returned content

In [41]:
# Getting response and checking status
resp = requests.get("http://api.open-notify.org/iss-pass.json", 
                    params = {"lat":40.71, "lon":-74, "n":10})
if resp.status_code == requests.codes.ok:
    print("Status OK")
else:
    print("Didn't Work")

Status OK


In [42]:
# Check the API and interpret your results - when will ISS pass over NEW York next ?
print(resp.text)
print(resp.headers)

{
  "message": "success", 
  "request": {
    "altitude": 100, 
    "datetime": 1575679175, 
    "latitude": 40.71, 
    "longitude": -74.0, 
    "passes": 10
  }, 
  "response": [
    {
      "duration": 567, 
      "risetime": 1575681337
    }, 
    {
      "duration": 602, 
      "risetime": 1575735670
    }, 
    {
      "duration": 641, 
      "risetime": 1575741445
    }, 
    {
      "duration": 573, 
      "risetime": 1575747324
    }, 
    {
      "duration": 571, 
      "risetime": 1575753192
    }, 
    {
      "duration": 642, 
      "risetime": 1575759004
    }, 
    {
      "duration": 616, 
      "risetime": 1575764814
    }, 
    {
      "duration": 545, 
      "risetime": 1575819202
    }, 
    {
      "duration": 649, 
      "risetime": 1575824931
    }, 
    {
      "duration": 590, 
      "risetime": 1575830793
    }
  ]
}

{'Server': 'nginx/1.10.3', 'Date': 'Sat, 07 Dec 2019 00:39:35 GMT', 'Content-Type': 'application/json', 'Content-Length': '855', 'Connection': '

In [48]:
import ast
import datetime as dt

d = ast.literal_eval(resp.text)
dt_code = d["response"][0]["risetime"]
dt_code = dt.datetime.fromtimestamp(dt_code)
dt_code

datetime.datetime(2019, 12, 6, 20, 15, 37)

According to the unix timecode given by the API, the ISS will be passing over New York next on December 6, 2019 at 6:38 PM. That is actually earlier today, so I'd like to see when the next few are.

In [49]:
codes = []
for idx, iss_pass in enumerate(d["response"]):
    code = iss_pass["risetime"]
    code = dt.datetime.fromtimestamp(code)
    codes.append(code)
for code in codes:
    print(code)

2019-12-06 20:15:37
2019-12-07 11:21:10
2019-12-07 12:57:25
2019-12-07 14:35:24
2019-12-07 16:13:12
2019-12-07 17:50:04
2019-12-07 19:26:54
2019-12-08 10:33:22
2019-12-08 12:08:51
2019-12-08 13:46:33


### Finding the number of people in space

OpenNotify has one more API endpoint, `/astros.json`. It tells you how many people are currently in space. The format of the responses can be studied [HERE](http://open-notify.org/Open-Notify-API/People-In-Space/).

Read the above documentation and perform the following tasks:

* Get the response from astros.json endpoint
* Count how many people are currently in space
* List the names of people currently in space.

In [50]:
# Get response and check status:
resp = requests.get("http://api.open-notify.org/astros.json")
if resp.status_code == requests.codes.ok:
    print("Status OK")
else:
    print("Didn't Work")

Status OK


In [51]:
# Interpret the Results - How many people are in space and what are their names 
print(resp.text)

{"people": [{"name": "Christina Koch", "craft": "ISS"}, {"name": "Alexander Skvortsov", "craft": "ISS"}, {"name": "Luca Parmitano", "craft": "ISS"}, {"name": "Andrew Morgan", "craft": "ISS"}, {"name": "Oleg Skripochka", "craft": "ISS"}, {"name": "Jessica Meir", "craft": "ISS"}], "number": 6, "message": "success"}


In [64]:
# Cleaning up the response:
names = []
d = ast.literal_eval(resp.text)
for astro in d["people"]:
    names.append(astro["name"])
print(f'There are {d["number"]} astronauts in space.\nTheir names are:')
for name in names:
      print(name)

There are 6 astronauts in space.
Their names are:
Christina Koch
Alexander Skvortsov
Luca Parmitano
Andrew Morgan
Oleg Skripochka
Jessica Meir


## Summary 

In this lesson, we saw how we can use request and response methods to query an Open API. We also saw how to look at the contents returned with the API calls and how to parse them. Next, we'll look at connecting to APIs which are not OPEN, i.e. we would need to pass in some authentication information and filter the results. 